<a href="https://colab.research.google.com/github/JigglyNoahPuff/CSE280/blob/main/Module5_Simplified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#### Basics of Building, Compiling, and Training a Deep Neural Network

### Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import math
import statistics as stats
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras import regularizers

### Reading in the data
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

In [ ]:
### Preparing the data
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
bikes = bikes.drop('dteday', axis = 1)

bikes['season'] = bikes['season'].map({1:'winter', 2:'spring', 3:'summer', 4:'fall'})
bikes['weathersit'] = bikes['weathersit'].map({1:'Great', 2:'Okay', 3:'Not_Good', 4:'Awful'})

bikes = pd.get_dummies(bikes, prefix='', prefix_sep='')
bikes['total_users'] = bikes['casual'] + bikes['registered']
bikes = bikes.drop(['casual', 'registered', 'Unnamed: 0', 'instant'], axis = 1)

bikes.head()

,hr,holiday,workingday,hum,windspeed,temp_c,feels_like_c,fall,spring,summer,winter,Awful,Great,Not_Good,Okay,total_users
0,0,0,0,0.81,0,3.28,3.0014,0,0,0,1,0,1,0,0,16
1,1,0,0,0.80,0,2.34,1.9982,0,0,0,1,0,1,0,0,40
2,2,0,0,0.80,0,2.34,1.9982,0,0,0,1,0,1,0,0,32
3,3,0,0,0.75,0,3.28,3.0014,0,0,0,1,0,1,0,0,13
4,4,0,0,0.75,0,3.28,3.0014,0,0,0,1,0,1,0,0,1


In [ ]:
train_dataset = bikes.sample(frac=0.8, random_state=0)
test_dataset = bikes.drop(train_dataset.index)

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('total_users')
test_labels = test_features.pop('total_users')

In [ ]:
### Creating a normalizer layer
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))

In [ ]:
### Building and Compiling the Model

# This is here because I got a warning otherwise
tf.keras.backend.set_floatx('float32')

# This is here as an optional method for calculating error and can be used as the loss function
def root_mean_squared_error(y_true,y_pred):
        y_true = float(y_true)
        y_pred = float(y_pred)
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

# Here we are building the model, each layer takes input from every node in the previous layer
#    First layer is input, last layer is output
dnn_model = keras.Sequential([
      normalizer,
      layers.Dense(512, activation='relu'),
      layers.Dense(512, activation='relu'),
      layers.Dense(1)
  ])

# Here we compile the model with the loss method and optimizer specified
dnn_model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.Adam(0.03))

combined_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(512,
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512,
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512,
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512,
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

combined_model.compile(loss=root_mean_squared_error, optimizer=tf.keras.optimizers.Adam(0.01))

In [ ]:
%%time
### Training the model to our train data
# %%time just outputs how long it took for the cell to finish

# Epochs can be adjusted to train for more or less
combined_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, batch_size= 1024, epochs=2048)

KeyboardInterrupt: ignored

In [ ]:
### Evaluating how well our model does
# This outputs our average loss across the test data
combined_model.evaluate(test_features, test_labels, verbose=0)

In [ ]:
### Showing how well our model performs on a graph

# Flattens the output into a single array
test_predictions = combined_model.predict(test_features).flatten()

# Plotting using matplotlib, and yes this cell was straight up copied from the regression tutorial
a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [Bikes Needed]')
plt.ylabel('Predictions [Bikes Needed]')
lims = [0, 1000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims, 'r')


In [ ]:
###Charging people for riding through dangerous areas

def extraCharge(rideActive, gpsEast, gpsNorth):
  extraCost = 0
  dangerCharge = 1.50
  topLeft = "116.2693192°E 39.9652585°N" 
  topRight = "116.4775480°E 39.9652585°N" 
  bottomRight = "116.4775480°E 39.8297188°N"
  bottomLeft = "116.2693192°E 39.8297188°N"
  if gpsEast < 116.4775480 and gpsEast > 116.2693192 and gpsNorth < 39.965285 and gpsNorth > 39.8297188:
    extraCost += dangerCharge
  return extraCost

cost = 0
rideActive = True
gpsEast = 115.2693192
gpsNorth = 38.8297188
totalCost = 0
time = 0
changeCost = [0]
changeTime = [0]
while rideActive:
  cost = cost + 0.1000000000 + extraCharge(rideActive, gpsEast, gpsNorth)
  changeCost.append(cost)
  gpsEast += 0.025
  gpsNorth += 0.025
  time += 1
  changeTime.append(time)
  if gpsNorth > 41 or gpsEast > 118:
    rideActive = False

plt.ylim([0,20])
plt.xlabel('Time [Minutes]')
plt.ylabel('Cost of Ride [Dollars]')
plt.plot(changeTime, changeCost)
print(changeCost)